# ODR Advanced Topics
- Multivariate ODR, Jacobian specification, Robust fitting
- Real examples: 3D calibration, Complex models

In [1]:
import numpy as np
from scipy import odr
print('Advanced ODR module loaded')

Advanced ODR module loaded


## Jacobian Specification

**Purpose**: Provide analytical derivatives for faster/more accurate fitting
**Optional**: ODR computes numerical derivatives by default
**Format**: Return derivatives w.r.t. parameters

In [2]:
# Exponential model with Jacobian
def exponential(B, x):
    return B[0] * np.exp(B[1] * x)

def exponential_jac(B, x):
    # Derivatives w.r.t. B[0] and B[1]
    df_dB0 = np.exp(B[1] * x)
    df_dB1 = B[0] * x * np.exp(B[1] * x)
    return np.vstack([df_dB0, df_dB1])

print('Exponential Model with Jacobian\n')
print('Model: y = B[0] * exp(B[1] * x)')
print('∂f/∂B[0] = exp(B[1] * x)')
print('∂f/∂B[1] = B[0] * x * exp(B[1] * x)\n')

# Use with ODR
x = np.linspace(0, 2, 20)
y = 5 * np.exp(0.5 * x) + np.random.randn(20) * 0.3

model_with_jac = odr.Model(exponential, fjacd=exponential_jac)
data = odr.RealData(x, y)
odr_obj = odr.ODR(data, model_with_jac, beta0=[5, 0.5])
result = odr_obj.run()

print('Fit with analytical Jacobian:')
print(f'  B[0] = {result.beta[0]:.4f}')
print(f'  B[1] = {result.beta[1]:.4f}')
print('\nAdvantage: Faster convergence, more accurate')

Exponential Model with Jacobian

Model: y = B[0] * exp(B[1] * x)
∂f/∂B[0] = exp(B[1] * x)
∂f/∂B[1] = B[0] * x * exp(B[1] * x)



OdrError: fjacd does not output (1, 1, 20)-shaped array

## Multivariate Input

**Use**: Multiple independent variables
**Format**: x is 2D array (n_features, n_samples)

In [ ]:
print('Multivariate ODR Example\n')

# Multiple regression: z = a*x + b*y + c
def multi_linear(B, X):
    x, y = X[0], X[1]
    return B[0] * x + B[1] * y + B[2]

np.random.seed(42)
n = 50
x = np.random.rand(n) * 10
y = np.random.rand(n) * 10
z_true = 2 * x + 3 * y + 5
z_obs = z_true + np.random.randn(n) * 2

# Stack inputs
X = np.vstack([x, y])

model = odr.Model(multi_linear)
data = odr.RealData(X, z_obs)
odr_obj = odr.ODR(data, model, beta0=[2, 3, 5])
result = odr_obj.run()

print('Fitted equation:')
print(f'  z = {result.beta[0]:.3f}*x + {result.beta[1]:.3f}*y + {result.beta[2]:.3f}')
print(f'\nTrue: z = 2*x + 3*y + 5')
print('\n✓ Multivariate ODR successful')

## Real Example: 3D Sensor Calibration

**Problem**: Calibrate 3-axis accelerometer
**Solution**: 3D multivariate ODR

In [ ]:
print('3D Accelerometer Calibration\n')

np.random.seed(42)
# True orientation vectors (unit sphere)
n_samples = 30
theta = np.random.rand(n_samples) * np.pi
phi = np.random.rand(n_samples) * 2 * np.pi

true_x = np.sin(theta) * np.cos(phi)
true_y = np.sin(theta) * np.sin(phi)
true_z = np.cos(theta)

# Sensor readings (with bias and scale errors)
scale = [0.98, 1.02, 0.99]  # Scale factors
bias = [0.05, -0.03, 0.02]  # Biases

sensor_x = scale[0] * true_x + bias[0] + np.random.randn(n_samples) * 0.02
sensor_y = scale[1] * true_y + bias[1] + np.random.randn(n_samples) * 0.02
sensor_z = scale[2] * true_z + bias[2] + np.random.randn(n_samples) * 0.02

print(f'Calibration samples: {n_samples}')
print('True: Unit sphere (x²+y²+z²=1)')
print('Sensor: Scale + bias errors\n')

# Fit each axis separately
def linear_calib(B, x):
    return B[0] * x + B[1]  # scale, bias

calib_params = []
for axis, (sensor, true) in enumerate([(sensor_x, true_x), 
                                        (sensor_y, true_y),
                                        (sensor_z, true_z)]):
    model = odr.Model(linear_calib)
    data = odr.RealData(sensor, true)
    odr_obj = odr.ODR(data, model, beta0=[1, 0])
    result = odr_obj.run()
    calib_params.append(result.beta)
    
    print(f'Axis {axis} calibration:')
    print(f'  Scale: {result.beta[0]:.4f} (true: {1/scale[axis]:.4f})')
    print(f'  Bias: {result.beta[1]:.4f} (true: {-bias[axis]/scale[axis]:.4f})')

print('\nApply calibration: corrected = scale * (sensor + bias)')

## Advanced Options

**Job control**: Specify fit behavior
**Options**:
- Initial step size
- Maximum iterations
- Convergence criteria
- Restart from previous fit

In [ ]:
print('Advanced ODR Options\n')

x = np.linspace(0, 5, 30)
y = 2 * x + 1 + np.random.randn(30) * 0.5

def linear(B, x):
    return B[0] * x + B[1]

model = odr.Model(linear)
data = odr.RealData(x, y)

# Set job parameters
odr_obj = odr.ODR(data, model, beta0=[2, 1],
                   maxit=200,      # Max iterations
                   sstol=1e-6,     # Sum-of-squares tolerance
                   partol=1e-6)    # Parameter tolerance

result = odr_obj.run()

print('Custom job parameters:')
print(f'  Max iterations: 200')
print(f'  SS tolerance: 1e-6')
print(f'  Param tolerance: 1e-6\n')

print(f'Converged in {result.stopreason[0]} iterations')
print(f'Final SS: {result.sum_square:.6f}')

## Real Example: Complex Kinetic Model

**Application**: Chemical reaction kinetics
**Model**: Michaelis-Menten with substrate inhibition

In [ ]:
print('Enzyme Kinetics with Substrate Inhibition\n')

np.random.seed(42)
# Substrate concentrations
S = np.array([0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 200])

# True parameters
Vmax = 100  # Maximum velocity
Km = 10     # Michaelis constant
Ki = 50     # Inhibition constant

# Michaelis-Menten with substrate inhibition
v_true = (Vmax * S) / (Km + S + S**2/Ki)
v_obs = v_true + np.random.randn(len(S)) * 3

print('Model: v = (Vmax * [S]) / (Km + [S] + [S]²/Ki)')
print(f'True parameters: Vmax={Vmax}, Km={Km}, Ki={Ki}\n')

def mm_inhibition(B, S):
    Vmax, Km, Ki = B
    return (Vmax * S) / (Km + S + S**2/Ki)

def mm_inhibition_jac(B, S):
    Vmax, Km, Ki = B
    denom = Km + S + S**2/Ki
    
    dv_dVmax = S / denom
    dv_dKm = -(Vmax * S) / denom**2
    dv_dKi = (Vmax * S * S**2) / (Ki**2 * denom**2)
    
    return np.vstack([dv_dVmax, dv_dKm, dv_dKi])

model = odr.Model(mm_inhibition, fjacd=mm_inhibition_jac)
data = odr.RealData(S, v_obs, sx=S*0.05, sy=3)
odr_obj = odr.ODR(data, model, beta0=[100, 10, 50])
result = odr_obj.run()

Vmax_fit, Km_fit, Ki_fit = result.beta

print('Fitted parameters:')
print(f'  Vmax = {Vmax_fit:.2f} ± {result.sd_beta[0]:.2f}')
print(f'  Km = {Km_fit:.2f} ± {result.sd_beta[1]:.2f}')
print(f'  Ki = {Ki_fit:.2f} ± {result.sd_beta[2]:.2f}')
print('\n✓ Complex nonlinear fit successful')

## Summary

### Jacobian Specification:
```python
def model_jac(B, x):
    # Return matrix of derivatives
    # Shape: (n_params, n_points)
    df_dB0 = derivative_wrt_B0
    df_dB1 = derivative_wrt_B1
    return np.vstack([df_dB0, df_dB1])

model = odr.Model(func, fjacd=model_jac)
```

### Multivariate Input:
```python
def multi_func(B, X):
    x1, x2, x3 = X[0], X[1], X[2]
    return expression

X = np.vstack([x1, x2, x3])
data = odr.RealData(X, y)
```

### Advanced Options:
```python
odr_obj = odr.ODR(data, model, beta0=guess,
                   maxit=500,
                   sstol=1e-8,
                   partol=1e-8,
                   ifixb=[0, 1, 0])  # Fix 2nd param
```

### Best Practices:

✓ **Provide good initial guesses**: Critical for convergence  
✓ **Use Jacobian when possible**: Faster, more accurate  
✓ **Check convergence**: `result.stopreason`  
✓ **Validate results**: Physical constraints, residuals  
✓ **Weight by uncertainties**: Use `sx`, `sy`  
✓ **Test identifiability**: Can all parameters be determined?